# SLU18 - Support Vector Machines

In this SLU, we will talk about another kind of classification algorithms, the support vector machines, developed in the 1990s and considered one of the best out-of-the-box classifiers.

<img src="media/into.png" width="300">

### Table of contents
[1. Introduction](#1.-Introduction)   
[2. Hyperplanes](#2.-Hyperplanes)   
[3. Maximal Margin Classifier](#3.-Maximal-Margin-Classifier)   
[4. Support Vector Classifier](#4.-Support-Vector-Classifier)   
&emsp;[4.1 More on the C Penalty](#4.1-More-on-the-C-Penalty)   
[5. Support Vector Machine](#5.-Support-Vector-Machine)   
&emsp;[5.1 Kernels](#5.1-Kernels)   
&emsp;[5.2 Polynomial kernel](#5.2-Polynomial-kernel)   
&emsp;[5.2 Radial kernel](#5.3-Radial-kernel)   
[6. Support Vector Regression](#6.-Support-Vector-Regression)   
[7. Final remarks](#7.-Final-remarks)   
&emsp;[7.1 References and further reading](#7.1-References-and-further-reading)

In [1]:
import warnings
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

warnings.simplefilter("ignore")

## 1. Introduction

We will use the Iris dataset, the same we used for logistic regression in SLU09. We divide it into train and test set and scale it. The data set has four features and 3 classes.

In [2]:
# load the iris dataset and train-test split it
X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# SVMs are not scale invariant, so we should scale our data beforehand
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [3]:
# Take a look at the first five observations' features

X_train[0:5, :]

array([[-0.87591581,  1.00478948, -1.3721624 , -1.21775109],
       [-0.28140282, -0.34998285, -0.10687818,  0.1058914 ],
       [-1.35152621,  0.32740331, -1.25713656, -1.35011534],
       [-1.47042881,  0.32740331, -1.3721624 , -1.35011534],
       [-0.75701322,  1.00478948, -1.31464948, -1.35011534]])

In [4]:
# Take a look at the first five observations' class
y_train[0:5]

array([0, 1, 0, 0, 0])

## 2. Hyperplanes

The SVM classifiers use hyperplanes to separate the observations into classes. Let's talk about what is a hyperplane. In a p-dimensional space, a hyperplane is a flat affine subspace of dimension p − 1 (affine means not passing through the origin). You already know some hyperplanes. In two dimensions, a hyperplane is a flat one-dimensional subspace — in other words, a line. In three dimensions, a hyperplane is a flat two-dimensional subspace — that is, a plane. In p > 3 dimensions, it becomes hard to visualize, but the notion of a (p − 1)-dimensional flat subspace still applies.

<img src="media/hyperplanes.png" width="800">

Consider a $n×p$ data matrix **X** that consists of n training observations in p-dimensional feature space. A hyperplane in the p-dimensional setting is defined by the below equation, 

$\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p} = 0$

Using this hyperplane, we can separate the observations into two classes. All observations on one side of the hyperplane that satisfy

$\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p} > 0$

will be classified as one class, while the observations on the other side that satisfy 

$\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p} < 0$

will be classified as the remaining class. 

## 3. Maximal Margin Classifier

The maximal margin classifier (MMC) is a binary classification method that makes use of the **optimal separating hyperplane**, which is the separating hyperplane that is farthest from the training observations. The minimal distance from the observations to the hyperplane is known as the **margin**. The observations that lie on the margin are known as the **support vectors**. The figure below illustrates the decision boundary (black line), the margin (dashed lines), and the support vectors (points on the dashed line).

<img src="media/mmc.png" width="400">

In order to construct the optimal separating hyperplane based on a set of n training observations $x_{1}, . . . , x_{n} ∈ R_{p}$ and associated class labels $y_{1}, . . . , y_{n} $∈ {-1,1}, we must find the solution to the optimization problem

* maximize $M$, parametrized by $\beta_{0},...,\beta_{p}$, subject to:

* $\sum_{j=1}^{p} β_j^{2}=1$

* $y_{i}(\beta_{0}+ \beta_{1}x_{i1}+ ... +\beta_{p}x_{ip}) \ge  M, \ \ \ ∀ i = 1, . . . , n$

Here, M represents the width of the margin, and the optimization problem chooses $\beta_{0}+ \beta_{1} + ... + \beta_{p}$ to maximize M.

The second constraint does not constrain the hyperplane, it just let's us express the distance of the observations from the hyperplane in a nice form as $y_{i}(\beta_{0}+ \beta_{1}x_{i1}+ ... +\beta_{p}x_{ip})$.

The support vectors "support" the separating hyperplane - if they move, the hyperplane also moves. MMC is very dependent on these few points that constitute the support vectors. If they move, it can affect the position of the hyperplane dramatically. On the other hand, moving any of the other observations does not affect the position of the hyperplane (unless we moves them into the margin).

The maximal margin classifier is a very natural way to perform classification if a separating hyperplane exists, that is, when the classes are well separated. However, in many cases the observations overlap and no separating hyperplane exists. MMC is also prone to overfitting in high dimesional feature spaces.

<img src="media/unseparable_meme.png" width="400">

## 4. Support Vector Classifier

The support vector classifier (SVC), also known as the soft margin classifier, is an extension of the maximal margin classifier. The soft margin allows some observations to be on the incorrect side of the margin, or even on the incorrect side of the hyperplane. 

This overcomes the MMC's limitation of not being able to handle cases where no separating hyperplane exists. In fact, even if a separating hyperplane does exist, there are instances in which a classifier based on such a separating hyperplane might not be desirable. An MMC classifier based on a separating hyperplane will necessarily perfectly classify all the training observations, which leads to a high sensitivity to individual observations.

By not perfectly separating the two classes, the SVC has greater robustness to individual observations and a better classification of *most* of the observations. The image below illustrates the decision boundary and margin of an SVC, 

<img src="media/svc.png" width="350">

As can be seen, some observations violate the margin. The method to obtain this hyperplane is similar to that of the MMC, except that we now introduce slack variables $s_{0}, ..., s_{n}$ and a non-negative tuning parameter C.

* maximize M, parametrized by $\beta_{0},...,\beta_{p}$, subject to:

* $\sum_{j=1}^{p} β_j^{2}=1$

* $y_{i}(\beta_{0}+ \beta_{1}x_{i1}+ ... +\beta_{p}x_{ip}) \ge M(1-s_{i}) $

* $s_{i} ≥ 0,\ \ \ \sum_{j=1}^{p} s_{j} ≤ C,\ \ \ ∀ i = 1, . . . , n\$

The slack variables $s_{0}, ..., s_{n}$  allow individual observations to be on the wrong side of the margin or the hyperplane. The magnitude of the slack variables controls the severity of the violation: $s_i$ > 0 allows the observation on the wrong side of the margin and $s_i$ > 1 on the wrong side of the hyperplane. With $s_i$ = 0, the observation is on the margin.

The C parameter is usually tuned in cross-validation and controls the bias-variance trade-off. C controls how many observations can be misclassified. C > 0 allows at most C observations on the wrong side of the hyperplane. C = 0 means no violations, which is the same as the MMC classifier.

Let's use the sklearn implementation of SVC. The classifier is inherently binary, but can be used with multiple classes through the one-vs-one or one-vs-rest strategies.

In the one-vs-one strategy, the classifier is sequentially fit to all pairs of classes. The final class of any given observation is the one most frequently assigned to it.

In the one-vs-rest strategy, we fit a classifier for each class with the data divided as inside or outside of the class. The observation is assigned to the class for which the distance to the hyperplane, $\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p}$, is the largest.

You can select the multiclass strategy with the `decision_function_shape` keyword set to `ovo` or `ovr` (default).

In [5]:
# Import the SVC class.
from sklearn.svm import SVC

# Create an estimator.
linear_svc = SVC(kernel="linear", C=1) # don't worry about the kernel argument for now
linear_svc

SVC(C=1, kernel='linear')

In [6]:
# Fit the estimator.
linear_svc.fit(X_train, y_train)
# Make predictions.
predictions = linear_svc.predict(X_test)
predictions

array([0, 2, 1, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 1, 1, 1, 2, 2, 1, 0, 1, 1,
       2, 0, 2, 2, 0, 0, 0, 1])

In [7]:
# The accuracy of the estimator.
linear_svc.score(X_test, y_test)

0.9666666666666667

In [8]:
# Get the support vectors.
linear_svc.support_vectors_

array([[-1.58933141, -1.70475519, -1.42967532, -1.21775109],
       [-0.87591581,  0.5531987 , -1.19962364, -0.95302259],
       [-0.87591581, -1.25316441, -0.45195569, -0.1588371 ],
       [ 0.55091538, -1.25316441,  0.64078978,  0.3706199 ],
       [ 0.55091538,  0.5531987 ,  0.52576394,  0.50298415],
       [ 0.19420758, -0.80157363,  0.75581562,  0.50298415],
       [ 0.55091538, -1.70475519,  0.35322518,  0.1058914 ],
       [ 1.14542837, -0.57577824,  0.58327686,  0.23825565],
       [-1.11372101, -1.4789598 , -0.27941693, -0.29120135],
       [ 1.26433097,  0.10160793,  0.64078978,  0.3706199 ],
       [ 0.07530498,  0.32740331,  0.58327686,  0.76771264],
       [ 1.02652577, -0.12418746,  0.6983027 ,  0.63534839],
       [-0.51920802, -0.12418746,  0.4107381 ,  0.3706199 ],
       [-0.28140282, -0.12418746,  0.4107381 ,  0.3706199 ],
       [ 0.78872058, -0.57577824,  0.46825102,  0.3706199 ],
       [-1.11372101, -1.25316441,  0.4107381 ,  0.63534839],
       [ 0.19420758, -0.

The training dataset has 120 observations, but only 26 of them are the support vectors and so influence the outcome of the classifier.

### 4.1 More on the C Penalty

The C penalty is the sum of all the slack variables, and thus determines the number and severity of the violations to the margin (and the hyperplane) that the model will tolerate.  If C=0 then we are back to the MMC. As C increases, the model becomes more tolerant and thus the margin will increase. This is illustrated by the plots below.

|![](media/small_C.png)|![](media/large_C.png)|
|:-:|:-:|
|small C|large C|

C controls the bias-variance trade-off of the classifier. It's basically a regularization parameter. When C is small, we get a narrow margin and less violations. The classifier is highly fit to the data, which may have low bias but high variance. When C is larger, the margin is wider and we allow more violations to it. This result is fitting the data less hard and obtaining a classifier that is potentially more biased but with lower variance.

<div class="alert alert-info">
    ⚠️ The C parameter in the sklearn SVC implementation is not the same. The strength of the regularization in sklearn is **inversely proportional** to C. The regularization is formulated differently. It is a L2 regularization, which means that the algorithm minimizes $C\sum_{j=1}^{p} s_{j}$ (as opposed to C being a constraint on the sum of the slack variables). The default value of C is 1. When C is small, then the margins will be wide and many support vectors will be on the margin or will violate the margin. When C is large, then the margins will be narrow and there will be few support vectors on the margin or violating the margin.
</div>    

## 5. Support Vector Machine

So far we have only considered models with a linear decision boundary. The support vector machine classifier (SVM) is the extension of the SVC to the non-linear case.

The general approach to fitting non-linear class boundaries is the inclusion of terms which are a non-linear function of the features, such as polynomials, into linear models as we did in SLU14. This approach works, but has a disadvantage - it becomes computationally expensive with too many features. SVM includes non-linear terms while being computationally efficient using kernels.

### 5.1 Kernels

Data not linearly separable in n-dimensional space may be linearly separable if we transform it into a higher dimensional space. 

![kernel_trick](media/kernel_trick.png)

However, performing computations in higher dimensions is more expensive. Kernels enlarge the feature space but still perform the calculations in the original lower dimensional space. This is known as the kernel trick.

Without going into technical details, the solution to the SVC classifier problem depends only on the inner products of the observations, not on the observations themselves:

$f(x) = \beta_0 + \sum \alpha_i ⟨x, xi⟩$

We can replace the inner product with a generalization of the inner product, some function which we call the kernel, $K(x_{i}, x_{n})$. A kernel is a function which computes the similarity of two observations.

A linear kernel, the dot product, gives us the SVC classifier with linear boundaries (do you remember the kernel argument in the sklearn SVC implementation?).

$K(x_{i}, x_{n}) =  \sum_{j=1}^{p}x_{ij}x_{nj}$ 

### 5.2 Polynomial kernel 
One popular non-linear kernel is the **polynomial kernel** of degree d, given by the equation below:

$K(x_{i}, x_{n}) = (1 + \sum_{j=1}^{p}x_{ij}x_{nj})^{d}$

Using such a kernel with d > 1 leads to a much more flexible decision boundary.

![poly_kernel](media/poly_kernel.png)

The sklearn implementation of SVM is the `SVC` class, but with non-linear kernels.

In [9]:
# Pass kernel='poly' and degree=d to create an
# SVM with a polynomial kernel of degree d.
polynomial_svm = SVC(kernel="poly", degree=3)
polynomial_svm

SVC(kernel='poly')

In [10]:
# Fit the model.
polynomial_svm.fit(X_train, y_train)
# Calculate the accuracy.
polynomial_svm.score(X_test, y_test)

0.9666666666666667

### 5.3 Radial kernel
Another popular option is the **radial kernel**, 

$K(x_{i}, x_{n}) = \exp(-\gamma\sum_{j=1}^{p}(x_{ij}-x_{nj})^{2})$

where  $\gamma$  is a positive constant. The below picture shows the same data classified using a radial kernel.

![rbf_kernel](media/rbf_kernel.png)

The sklearn implementation:

In [11]:
# Pass kernel='rbf' (default) to create an 
# SVM a with radial kernel.
radial_svm = SVC(kernel="rbf")
radial_svm

SVC()

In [12]:
# Fit the model.
radial_svm.fit(X_train, y_train)
# Calculate the accuracy.
radial_svm.score(X_test, y_test)

1.0

## 6. Support Vector Regression

The idea of a soft margin from the support vector classification can be extended to regression problems. This method is called support vector regression (SVR).

The support vector classification depends only on a subset of the training data, because the cost function of the model does not care about training points that lie beyond the margin. Analogously, support vector regression  model depends only on a subset of the training data. This is illustrated by the image below,

<img src="media/svr.png" width="400">

The standard linear regression minimizes the squared residuals of all observations. SVR also minimizes residuals, but the loss function includes only residuals larger in absolute values than some positive constant.

The sklearn implementation is [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html). Note that is also has the kernel argument, with `rbf` as default, and so is able to fit non-linear problems.

In [13]:
# Load the California housing dataset
from sklearn.datasets import fetch_california_housing
X, y = fetch_california_housing(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# Import the SVR estimator.
from sklearn.svm import SVR

# Create the SVR estimator.
svr = SVR()
svr

SVR()

In [14]:
# Fit the SVR estimator.
svr.fit(X_train, y_train)
# Make predictions.
svr.predict(X_test)

array([0.86026469, 1.25748412, 2.03040933, ..., 1.00095212, 4.93306971,
       0.81377737])

In [15]:
# Score the estimator (using R^2).
svr.score(X_test, y_test)

0.7311193579722226

## 7. Final remarks

That's it for support vector machines! Below is the recap of the main points of this SLU:
 
*   p-dimensional hyperplanes can be used as decision boundaries
*   The maximal margin classifier uses the optimal separating hyperplane
*   SVC uses a soft margin to allow misclassifications
*   SVM uses kernels to handle non-linearity
*   Extension to multi-class can be achieved with OVO and OVR
*   Extension to regression can be done with SVR

A final note: SVMs have a similar loss function as the logistic regression therefore they give similar results. Logistic regression also depends strongly on the observations close to the boundary and less so on the ones farther away. In comparision, SVMs perform better when the classes are separated and the logistic regression when classes overlap.

### 7.1 References and further reading

[The Kernel method](https://en.wikipedia.org/wiki/Kernel_method)

[Support Vector Regression](https://medium.com/coinmonks/support-vector-regression-or-svr-8eb3acf6d0ff)

Chapter 9 in Witten, Daniela, et al. [An Introduction to Statistical Learning](https://www.statlearning.com/), Springer, 2017.